In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import os
import pandas as pd
import numpy as np
import plotly.express as px


In [3]:
url=r'https://www.basketball-reference.com/friv/birthplaces.fcgi'
req = Request(url)
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

links=set(links)

In [4]:
foreign_links = [link for link in links if 'birthplaces' in link if 'country=US' not in link]


In [4]:
all_countries=[]
for link in foreign_links:
    
    url='https://www.basketball-reference.com'+link
    req = Request(url)
    html_page = urlopen(req)

    soup = BeautifulSoup(html_page)
    tables = soup.findAll('table')
    
    df = pd.read_html(tables[0].prettify())[0]
    country = soup.findAll('title')[0].text.split('Born in')[1].split('|')[0].strip()
    
    
    df.columns = df.columns.droplevel()
    df[df['Rk']!='Rk']
    
    df['Country']=country
    all_countries.append(df)

C:\Users\conno\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [5]:
col_names = ['Rk', 'Player', 'Yrs', 'From', 'To', 'G', 'MP_total', 'FG', 'FGA', '3P',
       '3PA', 'FT', 'FTA', 'ORB', 'TRB_total', 'AST_total', 'STL', 'BLK', 'TOV', 'PF',
       'PTS_total', 'FG%', '3P%', 'FT%', 'MP', 'PTS', 'TRB', 'AST', 'Date', 'City',
       'Country']

df = pd.concat(all_countries,axis=0, ignore_index=True)
df.columns = col_names
df.MP_total = df.MP_total.fillna(0)
df=df[df['Rk']!='Rk']
df.MP_total = df.MP_total.astype(float)
df.drop(['Rk'],axis=1,inplace=True) #drop rk column
df.drop(df.columns[[23,24,25,26]],axis=1,inplace=True) #drop per game columns, they have repeat names, recalculated easy if you want them

#fix misspellings
df.replace('Slavoska Pozega','Slavonska Pozega',inplace=True)
df.replace('Worzbach','Wurzbach',inplace=True)
df.replace('Eleuthra','Eleuthera',inplace=True)
df.replace('Albonito','Aibonito',inplace=True)
df.replace('Juiz de Forz','Juiz de Fora',inplace=True)
df.replace('Republic of Macedonia','North Macedonia',inplace=True)
#These next two return incorrect lat,lon with geopy. Including a comma after fixes this
df.replace('Buenos Aires','Buenos Aires,',inplace=True) 
df.replace('Valencia','Valencia,',inplace=True)


#drop players with no recorded hometown, 10 players are dropped
df.dropna(subset=['City'],inplace=True)

In [6]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="My_app")
latitudes=[]
longitudes=[]

for city,country in zip(df.City,df.Country):
    location = geolocator.geocode(str(city)+' '+str(country))
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)



In [13]:

df.To = df.To.astype(str)
df.To.replace('2020', "Active",inplace=True)
df['Latitude']=latitudes
df['Longitude']=longitudes
df.G = df.G.astype(int)
df.PTS_total = df.PTS_total.astype(int)
df.From = df.From.astype(int)
df.TRB_total = df.TRB_total.fillna(0).astype(int)
df.AST_total = df.AST_total.astype(int)
df.sort_values('From',inplace=True)
df['Decade'] = df.From.astype(str).str[:3]+'0\'s'
df['Minutes_per_game']=(df['MP_total']/df['G']).round(1)
df['Points_per_game'] = (df['PTS_total']/df['G']).round(1)
df['Assists_per_game'] = (df['AST_total']/df['G']).round(1)
df['Rebounds_per_game'] = (df['TRB_total']/df['G']).round(1)
df = df[df['From']<2020]
df = df[df['From']>1949]
df['Latitude']=df['Latitude']+np.random.randn(len(df))*0.1
df['Longitude']=df['Longitude']+np.random.randn(len(df))*0.1
df['Size']=[1 for x in range(len(df))]


In [60]:
df.to_csv('International_NBA.csv',index=False)